In [ ]:
!pip install PyPDF2
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 844.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import PyPDF2
import os
from sentence_transformers import SentenceTransformer, util

# Function to split the text into chunks
def split_into_chunks(text, max_chunk_size=200):
  """Splits the text into chunks of a maximum size."""
  chunks = []
  current_chunk = ""
  for sentence in text.split('.'):  # Split by sentences
    if len(current_chunk) + len(sentence) < max_chunk_size:
      current_chunk += sentence + '.'  # Add sentence to current chunk
    else:
      chunks.append(current_chunk.strip())  # Add current chunk to list
      current_chunk = sentence + '.'  # Start a new chunk
  chunks.append(current_chunk.strip())  # Add the last chunk
  return chunks

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
folder_path = '/content/drive/My Drive/PDFS'
all_chunks = []  # List to store chunks
document_sources = []  # List to store source PDF for each chunk, if multiple PDFs

In [ ]:
for filename in os.listdir(folder_path):
  if filename.endswith('.pdf'):
    pdf_path = os.path.join(folder_path, filename)
    with open(pdf_path, 'rb') as file:
      pdf_reader = PyPDF2.PdfReader(file)
      pdf_text = ""
      for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

      # Split PDF  into chunks
      chunks = split_into_chunks(pdf_text)
      all_chunks.extend(chunks)  # Add chunks to list
      document_sources.extend([filename] * len(chunks))  # Store source doc

In [ ]:
# Embed chunks
chunk_embeddings = model.encode(all_chunks)

In [ ]:
prompt = "What are the main methods to train an LLM without a lot of memory available?"
prompt_embedding = model.encode(prompt)
similarities = util.cos_sim(prompt_embedding, chunk_embeddings)

In [ ]:
# Get most similar chunk
most_similar_idx = similarities.argmax().item()
most_similar_chunk = all_chunks[most_similar_idx]
source_document = document_sources[most_similar_idx]

print(f"Prompt: {prompt}\nMost similar chunk: {most_similar_chunk}\nSource Document: {source_document}")

Prompt: What are the main methods to train an LLM without a lot of memory available?
Most similar chunk: com/@maxshapp/understanding-and-estimating-gpu-memory-demands-for-training-llms-in-practise-c5ef20a4baff
21 / 34Training: Mixed precision, cont
PyTorch’s AMP: ”Some ops, like linear layers ...
Source Document: SMU_Gen_AI_Topic_4.pdf


NOW LET'S LOOK AT USING CROSS_ENCODER https://www.sbert.net/examples/applications/cross-encoder/README.html

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

In [ ]:
pairs = []
for chunk in all_chunks:
    pairs.append([prompt, chunk])

In [ ]:
scores = cross_encoder.predict(pairs)

print(f"Prompt: {prompt}\n")
for chunk, score in zip(all_chunks, scores):
    print(f"Chunk: {chunk}\nScore: {score}\n")

Prompt: What are the main methods to train an LLM without a lot of memory available?

Chunk: Gen AI with LLMs Topic 4
Customising LLMs Efficiently
Laura Wynter, PhD
SMU School of Computing and Information Sciences
2025
1 / 34Recall the Motivation
▶Task-Specific objective: e.g.
Score: -9.185474395751953

Chunk: , medical or legal texts
▶Encoders are nearly always fine-tuned for specific objectives
▶Improved performance, for example better Q & A ability
▶Reducing Bias
▶Adding recent knowledge
2 / 34Plan for Lectures on Customising LLMs part II
▶The main libraries for LLM training, how and when to use them
▶Mixed precision & quantisation
▶PEFT
▶Distributed training (and inference)
3 / 34Main libraries for LLM training
▶Hugging Face
▶Torch’s FSDP
▶DeepSpeed from Microsoft
▶ollama (good library for group project)
4 / 34Hugging Face
▶Transformers is the main library within Hugging Face for training
▶Also, accelerate for distributed training
▶Also, PEFT for training in reduced parameter space

In [ ]:
import numpy as np
# Assuming 'scores' and 'all_chunks' are available from the previous cell
# Find the index of the chunk with the highest score
best_chunk_index = np.argmax(scores)
# Get the best chunk using the index
best_chunk = all_chunks[best_chunk_index]

# Print the best chunk
print(f"Best Chunk: {best_chunk}")

Best Chunk: The actual update uses the scaling
Wnew
0x=W0x+α
r(BA)x(a default is to use α=r)
▶Other hyper-parameters are less commonly modified, kept at defaults
19 / 34Training: Mixed precision
The main bottleneck for LLM training and fine-tuning is GPU memory
▶Industrial GPU like Nvidia A100 have 80 GB or 40 GB of memory, while
consumer-grade GPU may have 24 GB memory
▶Consider small GPT-2 model with 1.
